In [1]:
using PlotlyJS
using Parquet2: writefile, Dataset

include("../model/utils.jl")
include("../model/unit_commitment.jl")
include("../model/economic_dispatch.jl")
include("./plotting.jl")
include("./processing.jl")
mip_gap = 0.0001
reserve_margin = 0.1
min_reserve = 0

0

In [2]:

max_iterations = 1000
ed_config = Dict(:max_iterations => max_iterations)
days = [32]

configurations =  [:base_ramp_storage_reserve, :base_ramp_storage_envelopes, :base_ramp_storage_energy_reserve_cumulated]
;

In [3]:
s_uc = Dict()
s_ed = Dict()
for day in days, k in configurations
    gen_df, loads_multi_df, gen_variable_multi_df, storage_df, random_loads_multi_df = generate_input_data(day, "../input/base_case")
    required_reserve, required_energy_reserve, required_energy_reserve_cumulated = generate_reserves(loads_multi_df, gen_variable_multi_df, reserve_margin, min_reserve)
    
    configs = Dict(k => merge(v, ed_config) for (k,v) in generate_configurations(storage_df, required_reserve, required_energy_reserve, required_energy_reserve_cumulated))

    s_uc[(day,k)] = solve_unit_commitment(
        gen_df,
        loads_multi_df,
        gen_variable_multi_df,
        mip_gap;
        configs[k]...
    )
    s_ed[(day,k)] = solve_economic_dispatch(
        gen_df,
        random_loads_multi_df,
        gen_variable_multi_df,
        mip_gap;
        configs[k]...
    )
end
s_ed = merge_solutions(s_ed, [:day, :configuration])
s_uc = merge_solutions(s_uc, [:day, :configuration])

s_uc = Dict(pairs(s_uc))
s_uc[:reserve] = vcat(s_uc[:reserve], s_uc[:energy_reserve][s_uc[:energy_reserve].hour.==s_uc[:energy_reserve].hour_i,:][:,Not(:hour_i)])
s_uc = NamedTuple(s_uc)
;

Constructing UC...
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2475843
Academic license 2475843 - for non-commercial use only - registered to pa___@imperial.ac.uk
Set parameter MIPGap to value 0.01
Adding storage...
Adding ramp constraints...
Adding reserve constraints...
Set parameter MIPGap to value 0.01
Constructing UC...
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2475843
Academic license 2475843 - for non-commercial use only - registered to pa___@imperial.ac.uk
Set parameter MIPGap to value 0.01
Adding storage...
Adding ramp constraints...
Adding reserve constraints...
Set parameter MIPGap to value 0.01
Constructing EC...
Fixing decision variables...
SupplyDemand
SupplyDemandBalance
Cap_var
Solving ED...
LOLMax
SupplyDemandBalance
Cap_var
Solving ED...
LOLMax
SupplyDemandBalance
Cap_var
Solving ED...
LOLMax
SupplyDemandBalance
Cap_var
Solving ED...
LOLMax
SupplyDemandBalance
Cap_var
Solving ED...
LOLMax


In [ ]:
# using Parquet2: writefile, Dataset
write = false
read = false
if write
    using Parquet2: writefile, Dataset
    writefile("./notebooks/s_uc.parq", s_uc)
    writefile("./notebooks/s_ed.parq", s_ed)
end
if read
    s_uc = DataFrame(Dataset("./output/s_uc.parq"); copycols=false)
    s_ed = DataFrame(Dataset("./output/s_ed.parq"); copycols=false)  
end


In [ ]:
get_day = 1
gen_df, loads_multi_df, gen_variable_multi_df, storage_df, random_loads_multi_df = generate_input_data(days[get_day], "../input/base_case")
required_reserve, required_energy_reserve, required_energy_reserve_cumulated = generate_reserves(loads_multi_df, gen_variable_multi_df, reserve_margin, min_reserve)
;

In [ ]:
select_first = 100
to_plot_demand = stack(random_loads_multi_df[!,1:(select_first+1)], Not(:hour))
# to_plot_demand = s_ed.demand[s_ed.demand.day .== days[get_day],:]
to_plot_reserve = copy(required_reserve)
to_plot_reserve.reserve_up_MW = required_reserve.reserve_up_MW .+ random_loads_multi_df[!,:demand]
to_plot_reserve.reserve_down_MW = -required_reserve.reserve_down_MW .+ random_loads_multi_df[!,:demand]
to_plot_reserve.demand_MW = random_loads_multi_df[!,:demand]
to_plot_reserve = stack(to_plot_reserve, Not(:hour))
s= scatter(to_plot_demand, x = :hour, y = :value, group = :variable)
s1 = scatter(to_plot_reserve, x = :hour, y = :value, group = :variable, line=attr(color="purple", width=1.5, dash="dot"))
union!(s,s1)
p1 = plot(s, Layout(yaxis_title="power MW", xaxis_title="hour"))

t_ = loads_multi_df[loads_multi_df.demand .== maximum(loads_multi_df.demand),:hour]
to_plot_histogram = stack(random_loads_multi_df[random_loads_multi_df.hour .== t_, :])
p2 = plot(to_plot_histogram, x = :value, kind="histogram", histonorm = "percent", Layout(xaxis_title_text="demand at t = $t_"))

[p1 p2]

In [ ]:
thres =.001 # 1 Watt
f_LOL(x,y) = 
    (LOL_hours=count(x.>thres),
    LOLP=count(x.>thres)/length(y)*100,
    LOL_MWh = sum(x),
    LOL_percentage = sum(x)/(sum(y) + sum(x))*100,
    Demand_MWh = (sum(y) + sum(x)),
    )
f_CUR(x,y) =     
    (CUR_hours=count(x.>thres),
    CURP=count(x.>thres)/length(y)*100,
    CUR_MWh = sum(x),
    CUR_percentage = sum(x)/(sum(y) + sum(x))*100,
    RES_generation_MWh = (sum(y) + sum(x)),
    )
    

In [ ]:
group_by = [:configuration, :day]
LOL = combine(groupby(s_ed.demand, group_by), [:LOL_MW, :demand_MW] => ((x,y)->f_LOL(x,y)) => AsTable)
filter = in(["onshore_wind_turbine","small_hydroelectric","solar_photovoltaic", "net_generation"]).(s_ed.generation.resource)
CUR = combine(groupby(s_ed.generation[filter,:], group_by), [:curtailment_MW, :production_MW] =>((x,y) -> f_CUR(x,y))=> AsTable)
LOL_CUR = outerjoin(LOL, CUR, on=[:configuration, :day])
plot(LOL_CUR, x = :CURP, y = :LOLP, facet_row = :day, color = :configuration, mode = "markers")

In [ ]:
gdf_LOL = combine(groupby(s_ed.demand, union([:iteration], group_by)), [:LOL_MW, :demand_MW] => ((x,y)->f_LOL(x,y)) => AsTable)
plot(gdf_LOL, y = :LOLP, x = :LOL_MWh, facet_col = :configuration, facet_row = :day, color= :iteration, mode="markers")

In [ ]:
plot(gdf_CUR, y = :CURP, x = :CUR_MWh, facet_col = :configuration, facet_row = :day, color= :iteration, mode="markers")

In [ ]:
plot(gdf_LOL, x = :LOL_percentage, kind="histogram", facet_col = :configuration, facet_row = :day, histonorm = "percent")

In [ ]:
filter = in(["onshore_wind_turbine","small_hydroelectric","solar_photovoltaic"]).(s_ed.generation.resource) # probably not needed if we discard the CUR_percentage KPI
gdf_CUR = combine(groupby(s_ed.generation[filter,:], union([:iteration], group_by)), [:curtailment_MW, :production_MW] =>((x,y) -> f_CUR(x,y))=> AsTable)
plot(gdf_CUR, x = :CUR_percentage, kind="histogram", facet_col = :configuration, facet_row = :day, histonorm = "percent")

In [ ]:
unique(supply_ed.iteration)

In [ ]:
supply_uc, demand_uc = calculate_supply_demand(s_uc, union([:hour, :resource], group_by))
supply_ed, demand_ed = calculate_supply_demand(s_ed, union([:hour, :resource, :iteration], group_by))
;

In [ ]:
day_ = 32
iteration_ = :demand_3

In [ ]:
config_ = :base_ramp_storage_energy_reserve_cumulated
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))


In [ ]:
config_ = :base_ramp_storage_reserve
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))


In [ ]:
config_ = :base_ramp_storage_envelopes
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))

### Other

In [ ]:
plot_reserve(reserve_, "Storage + SOC Imprudent")

In [ ]:
supply_ = supply_uc[supply_uc.configuration .== :base_ramp_storage_envelopes, :]
demand_ = demand_uc[demand_uc.configuration .== :base_ramp_storage_envelopes, :]
solution_reserve_ = s_uc.reserve[s_uc.reserve.configuration .== :base_ramp_storage_envelopes, :]
reserve_ = calculate_reserve(solution_reserve_, required_reserve)
plot_supply_demand(supply_, demand_,"Storage + SOC Envelopes")

In [ ]:
plot_reserve(reserve_, "Storage + SOC Envelopes")

In [ ]:
supply_ = supply_uc[supply_uc.configuration .== :base_ramp_storage_energy_reserve_cumulated, :]
demand_ = demand_uc[demand_uc.configuration .== :base_ramp_storage_energy_reserve_cumulated, :]
solution_reserve_ = s_uc.reserve[s_uc.reserve.configuration .== :base_ramp_storage_energy_reserve_cumulated, :]
reserve_ = calculate_reserve(solution_reserve_, required_reserve)
plot_supply_demand(supply_, demand_,"Storage + Energy Reserve")

In [ ]:
plot_reserve(reserve_, "Storage + Energy Reserve")

## Other

In [ ]:
demand_ = :demand_28
supply_ed_ = supply_ed[(supply_ed.configuration .== :base_ramp_storage_envelopes).&(supply_ed.iteration .== demand_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== :base_ramp_storage_envelopes).&(demand_ed.iteration .== demand_), :]
# solution_reserve_ = s_uc.reserve[s_uc.reserve.configuration .== :base_ramp_storage_envelopes, :]
# reserve_ = calculate_reserve(solution_reserve_, required_reserve)
plot_supply_demand(supply_ed_, demand_ed_,"Storage + Envelopes")
# p1 = plot_fieldx_by_fieldy(supply_ed_, :production_MW, :resource, "ED - Storage + SOC Envelopes")

# p2 = plot_fieldx_by_fieldy(supply_ed_, :production_MW, :resource, "ED - Storage + Energy Reserve")

# [p1 p2]

In [ ]:
supply_ed_ = supply_ed[(supply_ed.configuration .== :base_ramp_storage_energy_reserve_cumulated).&(supply_ed.iteration .== demand_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== :base_ramp_storage_energy_reserve_cumulated).&(demand_ed.iteration .== demand_), :]
plot_supply_demand(supply_ed_, demand_ed_,"ED - Storage + Energy Reserve")